In [1]:
#install xml library
pip install lxml


The following command must be run outside of the IPython shell:

    $ pip install lxml

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
 #install request library
    pip install requests


The following command must be run outside of the IPython shell:

    $ pip install requests

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
#import library
import lxml.html as lh
import pandas as pd
import requests
 

In [6]:
#get url page content
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree =lh.fromstring(page.content)
tr_rows = tree.xpath('//tr')

In [7]:
#get table columns headings
col=[]
i=0

for t in tr_rows[0]:
    i+=1
    name=t.text_content()
    print  (i,name)
    col.append((name.rstrip('\n'),[]))

1 Postcode
2 Borough
3 Neighbourhood



In [10]:
#get table rows and create dataframe

for j in range(1,len(tr_rows)):
    T=tr_rows[j]
    if len(T)!=3:
         break
        #i is the index of our column
    i=0
     #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data.rstrip('\n'))
        #Increment i for the next column
        i+=1
            
[len(C) for (title,C) in col]   
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
# if neighbourhood is not assigned set to borough
for n in df.index: 
  if df['Neighbourhood'][n] == 'Not assigned':
     df['Neighbourhood'][n] = df['Borough'][n]
       
#keep row not equal to Not assigned
df2 = df[df.Neighbourhood != 'Not assigned']

#reset df index
df2.reset_index(inplace = True)
df2.drop(columns='index', inplace = True)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
for r in df2.index:
   df2['Neighbourhood'][r] = df2['Neighbourhood'][r] + ", "

df2_grp = df2.groupby(['Postcode','Borough'], as_index = False)['Neighbourhood'].agg(sum) 

#remove trailing ", "
for x in df2_grp.index:
     df2_grp['Neighbourhood'][x] = df2_grp['Neighbourhood'][x][0:len(df2_grp['Neighbourhood'][x])-2]

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
df2_grp.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df2_grp.shape

(103, 3)